In [19]:
!git clone https://github.com/yk03studios-wq/AnimeGANv3-fixed.git
%cd AnimeGANv3-fixed

In [20]:
!pip install -q onnxruntime-gpu tqdm Pillow
!pip install -q -r requirements.txt

In [21]:
!sed -i '/onnxruntime==/d' requirements.txt
!sed -i '/onnxruntime/d' requirements.txt
!sed -i '/tensorflow/d' requirements.txt

In [22]:
!pip install -q onnxruntime==1.22.1

In [23]:
!pip install -q -r requirements.txt

In [24]:
import onnxruntime as ort
print("ONNX Runtime:", ort.__version__)
print("Providers:", ort.get_available_providers())

In [ ]:
!python tools/video2anime.py -i inputs/vid/1.mp4 -o output/results -m deploy/AnimeGANv3_Hayao_36.onnx  

In [26]:
!ls -la output/results/

In [ ]:
import json
import subprocess
from pathlib import Path
from kaggle_secrets import UserSecretsClient

# === STEP 1: GET SECRETS + AUTH ===
user_secrets = UserSecretsClient()
username = user_secrets.get_secret("KAGGLE_USERNAME")
key = user_secrets.get_secret("KAGGLE_KEY")

if not username or not key:
    raise ValueError("❌ Missing KAGGLE_USERNAME or KAGGLE_KEY secrets")

print(f"✅ Authenticated as: {username}")

# Write kaggle.json
kaggle_dir = Path("/root/.config/kaggle")
kaggle_dir.mkdir(parents=True, exist_ok=True)
kaggle_json = kaggle_dir / "kaggle.json"

kaggle_json.write_text(json.dumps({"username": username, "key": key}, indent=2))
kaggle_json.chmod(0o600)

# Verify
result = subprocess.run(["kaggle", "config", "view"], capture_output=True, text=True)
print("🔍 CLI Config OK:", "username" in result.stdout)

# === STEP 2: PUBLISH (your existing prep code) ===
CANDIDATE_OUTPUT_DIRS = [
    Path("output/results"),
    Path("/kaggle/working/AnimeGANv3-fixed/output/results"),
]
OUTPUT_DIR = next((p for p in CANDIDATE_OUTPUT_DIRS if p.exists()), None)
DATASET_DIR = Path("/kaggle/working/dataset_export")
DATASET_DIR.mkdir(parents=True, exist_ok=True)
DATASET_ID = "yashkawade03/animegan-outputs-latest"

# Copy files + metadata (your existing code)
for file in OUTPUT_DIR.iterdir():
    if file.is_file():
        (DATASET_DIR / file.name).write_bytes(file.read_bytes())

dataset_metadata = {
    "title": "AnimeGAN Outputs (Latest)",
    "id": DATASET_ID,
    "licenses": [{"name": "CC0-1.0"}]
}
(DATASET_DIR / "dataset-metadata.json").write_text(json.dumps(dataset_metadata, indent=2))

print(f"📦 Ready: {DATASET_DIR}")

# === STEP 3: PUBLISH ===
for attempt, cmd in enumerate([
    # Update existing
    ["kaggle", "datasets", "version", "-p", str(DATASET_DIR), 
     "-m", "AnimeGANv3 latest - Feb 14, 2026", "--dir-mode", "zip"],
    # Create new  
    ["kaggle", "datasets", "create", "-p", str(DATASET_DIR), 
     "--dir-mode", "zip", "--public"]
], 1):
    
    print(f"\n🚀 Attempt {attempt}: {'Update' if attempt==1 else 'Create'}")
    print(" ".join(cmd))
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    print("OUT:", result.stdout.strip())
    
    if result.returncode == 0:
        print(f"✅ SUCCESS: https://www.kaggle.com/datasets/{DATASET_ID}")
        break
    else:
        print("ERR:", result.stderr.strip())
        if attempt == 1: print("↻ Retrying as new dataset...")
